# Calculate Stats on Tuning Distributions

In [5]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scikit_posthocs as posthoc
from statsmodels.stats.multitest import multipletests
from hmmlearn import hmm

sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import functions_plotting as fp

In [6]:
def h5_to_dict(filepath):
    with pd.HDFStore(filepath, 'r') as store:
        keys = store.keys()
        data = {key[1:]: store[key] for key in keys}
    return data


def kruskal_w_dunn(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.kruskal(*data, nan_policy=nan_policy)
    dunn_test = posthoc.posthoc_dunn([*data], p_adjust='holm')
    return p_val, dunn_test


def mannwhitneyu_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.mannwhitneyu(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def wilcoxon_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.wilcoxon(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def sample_df(data1, data2):
    df1 = data1.dropna()
    df2 = data2.dropna() 

    if len(df1) > len(df2):
        df1 = df1.sample(len(df2), replace=True)
    elif len(df2) > len(df1):
        df2 = df2.sample(len(df1), replace=True)
    else:
        pass

    return df1, df2


def bootstrap_tests(datasets, key, tuning_kind, method, n_iters=10000, alpha=0.05, bootstrap_all=False):
    data = [ds[key][tuning_kind].dropna() for ds in datasets]

    # First run the test on the raw data
    if 'wilcoxon' in method.__name__:
        raw_test = method(sample_df(*data), alpha=alpha)
    elif 'mannwhitneyu' in method.__name__:
        raw_test = method(data, alpha=alpha)
    else:
        raw_test = method(data, alpha=alpha)

    # Now run the bootstrapping
    p_vals = np.zeros(n_iters)
    p_vals_corrected = []

    sample_size = len(max(data, key=len))

    # Run bootstrap
    for i in range(n_iters):

        if bootstrap_all:
            sample_data = [d.sample(sample_size, replace=True) for d in data]
            # sample_data1  = data2.sample(sample_size, replace=True)
        else:
            sample_data = [data[0]] + [d.sample(sample_size, replace=True) for d in data[1:]]
            
        p_val, p_val_corrected = method(sample_data, alpha=alpha)
        p_vals[i] = p_val
        p_vals_corrected.append(p_val_corrected)

    # 95% confidence interval
    ci = stats.t.interval(0.95, len(p_vals) - 1, loc=np.nanmean(p_vals), scale=stats.sem(p_vals, nan_policy='omit'))

    if method == kruskal_w_dunn:
        ci_corrected = None
    else:
        ci_corrected = stats.t.interval(0.95, len(p_vals_corrected) - 1, loc=np.nanmean(p_vals_corrected), scale=stats.sem(p_vals_corrected, nan_policy='omit'))

    return raw_test, (p_vals, ci), (p_vals_corrected, ci_corrected)


def plot_bootstrap_distributions(data):
    # Plot the bootstrap distributions
    num_ds = len(data)
    cols = 3
    rows = num_ds//cols + 1
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))

    for i, (ds, ax) in enumerate(zip(data, axes.ravel())):

        raw_test = ds[0]
        bootstrap_tests = ds[1]
        corrected_bootstrap_tests = ds[2]

        ax.hist(bootstrap_tests[0], bins=20)
        ax.axvline(np.percentile(bootstrap_tests[0], 95), 0, 50, color='r')
        ax.axvline(bootstrap_tests[1][0], 0, 50, color='g')
        ax.axvline(bootstrap_tests[1][1], 0, 50, color='g')
        ax.axvline(raw_test[0], 0, 100, color='k')


    return fig, axes

In [7]:
basedir = r"Z:\Prey_capture\WF_Figures"
still_or_all = 'full'
main_exp_path = os.path.join(basedir, still_or_all, r"multi_normal_ALL\stats.hdf5")
control_light_path  = os.path.join(basedir, still_or_all, r"control_normal_ALL\stats.hdf5")
control_dark_path = os.path.join(basedir, still_or_all, r"control_dark_ALL\stats.hdf5")
repeat_fixed_path = os.path.join(basedir, still_or_all, r"repeat_normal_VWheelWF\stats.hdf5")
repeat_free_path = os.path.join(basedir, still_or_all, r"repeat_normal_VTuningWF\stats.hdf5")
output_path = os.path.join(basedir, 'stats', f"{still_or_all}_tuning_stats.h5")

In [8]:
main_exp_data = h5_to_dict(main_exp_path)
control_light_data = h5_to_dict(control_light_path)
control_dark_data = h5_to_dict(control_dark_path)
repeat_fixed_data = h5_to_dict(repeat_fixed_path)
repeat_free_data = h5_to_dict(repeat_free_path)

In [9]:
method = kruskal_w_dunn
bootstrap_iters = 1000

# Kinematic Tuning

In [10]:
def kinem_stats(data, ds_name, alpha=0.05):
    cols = data[0][ds_name].columns.to_list()

    stats = {}
    for col in cols:
        stats[col] = bootstrap_tests(data, ds_name, col,   
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)

    return stats

In [ ]:
kinem_tuning_fixed

In [11]:
# Fixed data
fixed_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_fixed')
kinem_tuning_fixed = pd.DataFrame(fixed_kinem).T
kinem_tuning_fixed.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_kinem_tuning_fixed'] = kinem_tuning_fixed


# Free data
free_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_free')
kinem_tuning_free = pd.DataFrame(free_kinem).T
kinem_tuning_free.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_kinem_tuning_free'] = kinem_tuning_free


C:\Users\mmccann\AppData\Local\Temp\ipykernel_16964\3641501808.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['main', 'control_light', 'control_dark'], dtype='object')]

  store['kruskal_kinem_tuning_fixed'] = kinem_tuning_fixed


KeyboardInterrupt: 

# Visual Tuning

## Frac Visual Tuning - Fixed to Free

### Kruskal-Wallis

In [ ]:
# Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons

# General
exp_fixed_free_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].general, 
                                              main_exp_data['frac_vis_resp_free'].general))
print('Fixed vs Free General:', exp_fixed_free_gen)

# Visual
exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].visual, 
                                              main_exp_data['frac_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

exp_fixed_free_vis_no_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].visual_no_gen, 
                                                     main_exp_data['frac_vis_resp_free'].visual_no_gen))
print('Fixed vs Free Visual No Gen:', exp_fixed_free_vis_no_gen)

exp_fixed_vis_vis_no_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].visual, 
                                                    main_exp_data['frac_vis_resp_fixed'].visual_no_gen))
print('Fixed Visual - Visual No Gen:', exp_fixed_vis_vis_no_gen)

exp_free_vis_vis_no_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_free'].visual, 
                                                   main_exp_data['frac_vis_resp_free'].visual_no_gen))
print('Free Visual - Visual No Gen:', exp_free_vis_vis_no_gen)

# Orientation
exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation, 
                                              main_exp_data['frac_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)

exp_fixed_free_ori_no_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation_no_gen, 
                                                     main_exp_data['frac_vis_resp_free'].orientation_no_gen))
print('Fixed vs Free Orientation No Gen:', exp_fixed_free_ori_no_gen)

exp_fixed_ori_ori_no_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation, 
                                                    main_exp_data['frac_vis_resp_fixed'].orientation_no_gen))
print('Fixed Orientation - Orientation No Gen:', exp_fixed_ori_ori_no_gen)

exp_free_ori_ori_no_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_free'].orientation, 
                                                    main_exp_data['frac_vis_resp_free'].orientation_no_gen))
print('Free Orientation - Orientation No Gen:', exp_free_ori_ori_no_gen)

# Direction
exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].direction, 
                                              main_exp_data['frac_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

exp_fixed_free_dir_no_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].direction_no_gen, 
                                                     main_exp_data['frac_vis_resp_free'].direction_no_gen))
print('Fixed vs Free Direction No Gen:', exp_fixed_free_dir_no_gen)

exp_fixed_dir_dir_no_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'].direction, 
                                                    main_exp_data['frac_vis_resp_fixed'].direction_no_gen))
print('Fixed Direction - Direction No Gen:', exp_fixed_dir_dir_no_gen)

exp_free_dir_dir_no_gen = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_free'].direction, 
                                                    main_exp_data['frac_vis_resp_free'].direction_no_gen))
print('Free Direction - Direction No Gen:', exp_free_dir_dir_no_gen)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_exp_fixed_free_gen'] = pd.DataFrame(exp_fixed_free_gen, index=['statistic', 'p_val'])
    store['kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['statistic', 'p_val'])
    store['kruskal_exp_fixed_free_vis_no_gen'] = pd.DataFrame(exp_fixed_free_vis_no_gen, index=['statistic', 'p_val'])
    store['kruskal_exp_fixed_vis_vis_no_gen'] = pd.DataFrame(exp_fixed_vis_vis_no_gen, index=['statistic', 'p_val'])
    store['kruskal_exp_free_vis_vis_no_gen'] = pd.DataFrame(exp_free_vis_vis_no_gen, index=['statistic', 'p_val'])
    store['kruskal_exp_fixed_freew_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['statistic', 'p_val'])
    store['kruskal_exp_fixed_free_ori_no_gen'] = pd.DataFrame(exp_fixed_free_ori_no_gen, index=['statistic', 'p_val'])
    store['kruskal_exp_fixed_ori_ori_no_gen'] = pd.DataFrame(exp_fixed_ori_ori_no_gen, index=['statistic', 'p_val'])
    store['kruskal_exp_free_ori_ori_no_gen'] = pd.DataFrame(exp_free_ori_ori_no_gen, index=['statistic', 'p_val'])
    store['kruskal_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['statistic', 'p_val'])
    store['kruskal_exp_fixed_free_dir_no_gen'] = pd.DataFrame(exp_fixed_free_dir_no_gen, index=['statistic', 'p_val'])
    store['kruskal_exp_fixed_dir_dir_no_gen'] = pd.DataFrame(exp_fixed_dir_dir_no_gen, index=['statistic', 'p_val'])
    store['kruskal_exp_free_dir_dir_no_gen'] = pd.DataFrame(exp_free_dir_dir_no_gen, index=['statistic', 'p_val'])

### Wilcoxon Rank Sum

In [ ]:
# Do a Wilcox Rank-Sum test with Bonferroni correction

# General
exp_fixed_free_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].general, 
                                              main_exp_data['frac_vis_resp_free'].general))
print('Fixed vs Free General:', exp_fixed_free_gen)

# Visual
exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].visual, 
                                              main_exp_data['frac_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

exp_fixed_free_vis_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].visual_no_gen, 
                                                     main_exp_data['frac_vis_resp_free'].visual_no_gen))
print('Fixed vs Free Visual No Gen:', exp_fixed_free_vis_no_gen)

exp_fixed_vis_vis_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].visual, 
                                                    main_exp_data['frac_vis_resp_fixed'].visual_no_gen))
print('Fixed Visual - Visual No Gen:', exp_fixed_vis_vis_no_gen)

exp_free_vis_vis_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_free'].visual, 
                                                   main_exp_data['frac_vis_resp_free'].visual_no_gen))
print('Free Visual - Visual No Gen:', exp_free_vis_vis_no_gen)

# Orientation
exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation, 
                                              main_exp_data['frac_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)

exp_fixed_free_ori_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation_no_gen, 
                                                     main_exp_data['frac_vis_resp_free'].orientation_no_gen))
print('Fixed vs Free Orientation No Gen:', exp_fixed_free_ori_no_gen)

exp_fixed_ori_ori_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation, 
                                                    main_exp_data['frac_vis_resp_fixed'].orientation_no_gen))
print('Fixed Orientation - Orientation No Gen:', exp_fixed_ori_ori_no_gen)

exp_free_ori_ori_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_free'].orientation, 
                                                    main_exp_data['frac_vis_resp_free'].orientation_no_gen))
print('Free Orientation - Orientation No Gen:', exp_free_ori_ori_no_gen)

# Direction
exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].direction, 
                                              main_exp_data['frac_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

exp_fixed_free_dir_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].direction_no_gen, 
                                                     main_exp_data['frac_vis_resp_free'].direction_no_gen))
print('Fixed vs Free Direction No Gen:', exp_fixed_free_dir_no_gen)

exp_fixed_dir_dir_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].direction, 
                                                    main_exp_data['frac_vis_resp_fixed'].direction_no_gen))
print('Fixed Direction - Direction No Gen:', exp_fixed_dir_dir_no_gen)

exp_free_dir_dir_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_free'].direction, 
                                                    main_exp_data['frac_vis_resp_free'].direction_no_gen))
print('Free Direction - Direction No Gen:', exp_free_dir_dir_no_gen)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['wilcoxon_exp_fixed_free_gen'] = pd.Series(exp_fixed_free_gen)
    store['wilcoxon_exp_fixed_free_vis'] = pd.Series(exp_fixed_free_vis)
    store['wilcoxon_exp_fixed_free_vis_no_gen'] = pd.Series(exp_fixed_free_vis_no_gen)
    store['wilcoxon_exp_fixed_vis_vis_no_gen'] = pd.Series(exp_fixed_vis_vis_no_gen)
    store['wilcoxon_exp_free_vis_vis_no_gen'] = pd.Series(exp_free_vis_vis_no_gen)
    store['wilcoxon_exp_fixed_free_ori'] = pd.Series(exp_fixed_free_ori)
    store['wilcoxon_exp_fixed_free_ori_no_gen'] = pd.Series(exp_fixed_free_ori_no_gen)
    store['wilcoxon_exp_fixed_ori_ori_no_gen'] = pd.Series(exp_fixed_ori_ori_no_gen)
    store['wilcoxon_exp_free_ori_ori_no_gen'] = pd.Series(exp_free_ori_ori_no_gen)
    store['wilcoxon_exp_fixed_free_dir'] = pd.Series(exp_fixed_free_dir)
    store['wilcoxon_exp_fixed_free_dir_no_gen'] = pd.Series(exp_fixed_free_dir_no_gen)
    store['wilcoxon_exp_fixed_dir_dir_no_gen'] = pd.Series(exp_fixed_dir_dir_no_gen)
    store['wilcoxon_exp_free_dir_dir_no_gen'] = pd.Series(exp_free_dir_dir_no_gen)

### Mann-Whitney U

In [ ]:
# Do a Mann-Whitney U test with Bonferroni correction

# General
exp_fixed_free_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].general, 
                                              main_exp_data['frac_vis_resp_free'].general))
print('Fixed vs Free General:', exp_fixed_free_gen)

# Visual
exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].visual, 
                                              main_exp_data['frac_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

exp_fixed_free_vis_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].visual_no_gen, 
                                                     main_exp_data['frac_vis_resp_free'].visual_no_gen))
print('Fixed vs Free Visual No Gen:', exp_fixed_free_vis_no_gen)

exp_fixed_vis_vis_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].visual, 
                                                    main_exp_data['frac_vis_resp_fixed'].visual_no_gen))
print('Fixed Visual - Visual No Gen:', exp_fixed_vis_vis_no_gen)

exp_free_vis_vis_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_free'].visual, 
                                                   main_exp_data['frac_vis_resp_free'].visual_no_gen))
print('Free Visual - Visual No Gen:', exp_free_vis_vis_no_gen)

# Orientation
exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation, 
                                              main_exp_data['frac_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)

exp_fixed_free_ori_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation_no_gen, 
                                                     main_exp_data['frac_vis_resp_free'].orientation_no_gen))
print('Fixed vs Free Orientation No Gen:', exp_fixed_free_ori_no_gen)

exp_fixed_ori_ori_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].orientation, 
                                                    main_exp_data['frac_vis_resp_fixed'].orientation_no_gen))
print('Fixed Orientation - Orientation No Gen:', exp_fixed_ori_ori_no_gen)

exp_free_ori_ori_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_free'].orientation, 
                                                    main_exp_data['frac_vis_resp_free'].orientation_no_gen))
print('Free Orientation - Orientation No Gen:', exp_free_ori_ori_no_gen)

# Direction
exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].direction, 
                                              main_exp_data['frac_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

exp_fixed_free_dir_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].direction_no_gen, 
                                                     main_exp_data['frac_vis_resp_free'].direction_no_gen))
print('Fixed vs Free Direction No Gen:', exp_fixed_free_dir_no_gen)

exp_fixed_dir_dir_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'].direction, 
                                                    main_exp_data['frac_vis_resp_fixed'].direction_no_gen))
print('Fixed Direction - Direction No Gen:', exp_fixed_dir_dir_no_gen)

exp_free_dir_dir_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_free'].direction, 
                                                    main_exp_data['frac_vis_resp_free'].direction_no_gen))
print('Free Direction - Direction No Gen:', exp_free_dir_dir_no_gen)

print('Free Direction - Direction No Gen:', exp_free_dir_dir_no_gen)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['mannwhitneyu_exp_fixed_free_gen'] = pd.Series(exp_fixed_free_gen)
    store['mannwhitneyu_exp_fixed_free_vis'] = pd.Series(exp_fixed_free_vis)
    store['mannwhitneyu_exp_fixed_free_vis_no_gen'] = pd.Series(exp_fixed_free_vis_no_gen)
    store['mannwhitneyu_exp_fixed_vis_vis_no_gen'] = pd.Series(exp_fixed_vis_vis_no_gen)
    store['mannwhitneyu_exp_free_vis_vis_no_gen'] = pd.Series(exp_free_vis_vis_no_gen)
    store['mannwhitneyu_exp_fixed_free_ori'] = pd.Series(exp_fixed_free_ori)
    store['mannwhitneyu_exp_fixed_free_ori_no_gen'] = pd.Series(exp_fixed_free_ori_no_gen)
    store['mannwhitneyu_exp_fixed_ori_ori_no_gen'] = pd.Series(exp_fixed_ori_ori_no_gen)
    store['mannwhitneyu_exp_free_ori_ori_no_gen'] = pd.Series(exp_free_ori_ori_no_gen)
    store['mannwhitneyu_exp_fixed_free_dir'] = pd.Series(exp_fixed_free_dir)
    store['mannwhitneyu_exp_fixed_free_dir_no_gen'] = pd.Series(exp_fixed_free_dir_no_gen)
    store['mannwhitneyu_exp_fixed_dir_dir_no_gen'] = pd.Series(exp_fixed_dir_dir_no_gen)
    store['mannwhitneyu_exp_free_dir_dir_no_gen'] = pd.Series(exp_free_dir_dir_no_gen)

## Frac Visual Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [12]:
# Do a Kruskal-Wallis test with Dunn posthoc

# General
frac_fixed_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'frac_vis_resp_fixed', 'general',   
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls General 95% CI:', frac_fixed_gen[1][1])
print('Fixed vs Controls General Raw:', frac_fixed_gen[0])

# Visual
frac_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'frac_vis_resp_fixed', 'visual',   
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Visual 95% CI:', frac_fixed_vis[1][1])
print('Fixed vs Controls Visual Raw:', frac_fixed_vis[0])

frac_fixed_vis_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_vis_resp_fixed', 'visual_no_gen',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Visual No Gen 95% CI:', frac_fixed_vis_no_gen[1][1])
print('Fixed vs Controls Visual No Gen Raw:', frac_fixed_vis_no_gen[0])

# Orientation
frac_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'frac_vis_resp_fixed', 'orientation',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Orientation 95% CI:', frac_fixed_ori[1][1])
print('Fixed vs Controls Orientation Raw:', frac_fixed_ori[0])

frac_fixed_ori_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_vis_resp_fixed', 'orientation_no_gen',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Orientation No Gen 95% CI:', frac_fixed_ori_no_gen[1][1])
print('Fixed vs Controls Orientation No Gen Raw:', frac_fixed_ori_no_gen[0])

# Direction
frac_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'frac_vis_resp_fixed', 'direction',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Direction 95% CI:', frac_fixed_dir[1][1])
print('Fixed vs Controls Direction Raw:', frac_fixed_dir[0])

frac_fixed_dir_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_vis_resp_fixed', 'direction_no_gen',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Direction No Gen 95% CI:', frac_fixed_dir_no_gen[1][1])
print('Fixed vs Controls Direction No Gen Raw:', frac_fixed_dir_no_gen[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_frac_fixed_gen'] = pd.DataFrame(list(frac_fixed_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_fixed_vis_no_gen'] = pd.DataFrame(list(frac_fixed_vis_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_fixed_ori_no_gen'] = pd.DataFrame(list(frac_fixed_ori_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_fixed_dir'] = pd.DataFrame(list(frac_fixed_dir), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_fixed_dir_no_gen'] = pd.DataFrame(list(frac_fixed_dir_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Fixed vs Controls General 95% CI: (-3.4111508604375425e-23, 1.4184337000505386e-22)
Fixed vs Controls General Raw: (2.909971363544586e-07,           1         2         3
1  1.000000  0.000111  0.000079
2  0.000111  1.000000  0.863954
3  0.000079  0.863954  1.000000)
Fixed vs Controls Visual 95% CI: (1.9923945486207725e-06, 1.126523616275191e-05)
Fixed vs Controls Visual Raw: (0.004131104489420829,           1         2        3
1  1.000000  0.008945  0.13291
2  0.008945  1.000000  0.30122
3  0.132910  0.301220  1.00000)
Fixed vs Controls Visual No Gen 95% CI: (-6.306195829635221e-29, 7.4017663842515785e-28)
Fixed vs Controls Visual No Gen Raw: (1.3170976430908415e-07,           1         2         3
1  1.000000  0.000119  0.000029
2  0.000119  1.000000  1.000000
3  0.000029  1.000000  1.000000)
Fixed vs Controls Orientation 95% CI: (0.00031046143692441494, 0.0008145635455459432)
Fixed vs Controls Orientation Raw: (0.015248188377096284,           1         2         3
1  1.000000  0.22

C:\Users\mmccann\AppData\Local\Temp\ipykernel_16964\4177184402.py:51: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_frac_fixed_gen'] = pd.DataFrame(list(frac_fixed_gen), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_16964\4177184402.py:53: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store['kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_16964\4177184402.py:55: PerformanceWarning: 
your performance may suffer as PyTables will pickle objec

In [ ]:
frac_fixed_fig = plot_bootstrap_distributions([frac_fixed_gen, 
                                               frac_fixed_vis, frac_fixed_vis_no_gen, 
                                               frac_fixed_ori, frac_fixed_ori_no_gen,
                                               frac_fixed_dir, frac_fixed_dir_no_gen])
plt.show()

### Free Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc

# General
frac_free_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_vis_resp_free', 'general',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls General 95% CI:', frac_free_gen[1][1])
print('Free vs Controls General Raw:', frac_free_gen[0])

# Visual
frac_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                  'frac_vis_resp_free', 'visual',
                                  kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Visual 95% CI:', frac_free_vis[1][1])
print('Free vs Controls Visual Raw:', frac_free_vis[0])

frac_free_vis_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                         'frac_vis_resp_free', 'visual_no_gen',
                                         kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Visual No Gen 95% CI:', frac_free_vis_no_gen[1][1])
print('Free vs Controls Visual No Gen Raw:', frac_free_vis_no_gen[0])

# Orientation
frac_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                  'frac_vis_resp_free', 'orientation',
                                  kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Orientation 95% CI:', frac_free_ori[1][1])
print('Free vs Controls Orientation Raw:', frac_free_ori[0])

frac_free_ori_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                         'frac_vis_resp_free', 'orientation_no_gen',
                                         kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Orientation No Gen 95% CI:', frac_free_ori_no_gen[1][1])
print('Free vs Controls Orientation No Gen Raw:', frac_free_ori_no_gen[0])

# Direction
frac_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                  'frac_vis_resp_free', 'direction',
                                  kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Direction 95% CI:', frac_free_dir[1][1])
print('Free vs Controls Direction Raw:', frac_free_dir[0])

frac_free_dir_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                         'frac_vis_resp_free', 'direction_no_gen',
                                          kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Direction No Gen 95% CI:', frac_free_dir_no_gen[1][1])
print('Free vs Controls Direction No Gen Raw:', frac_free_dir_no_gen[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_frac_free_gen'] = pd.DataFrame(list(frac_free_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_free_vis_no_gen'] = pd.DataFrame(list(frac_free_vis_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_free_ori_no_gen'] = pd.DataFrame(list(frac_free_ori_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_free_dir'] = pd.DataFrame(list(frac_free_dir), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_frac_free_dir_no_gen'] = pd.DataFrame(list(frac_free_dir_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T


In [ ]:
frac_free_fig = plot_bootstrap_distributions([frac_free_gen, 
                                              frac_free_vis, frac_free_vis_no_gen, 
                                              frac_free_ori, frac_free_ori_no_gen,
                                              frac_free_dir, frac_free_dir_no_gen])
plt.show()

## Count Vis Tuning - Fixed vs Free

### Kruskal-Wallis

In [ ]:
# Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons

# General
exp_fixed_free_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].general, 
                                              main_exp_data['count_vis_resp_free'].general))
print('Fixed vs Free General:', exp_fixed_free_gen)

# Visual
exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].visual, 
                                              main_exp_data['count_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

exp_fixed_free_vis_no_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].visual_no_gen, 
                                                     main_exp_data['count_vis_resp_free'].visual_no_gen))
print('Fixed vs Free Visual No Gen:', exp_fixed_free_vis_no_gen)

exp_fixed_vis_vis_no_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].visual, 
                                                    main_exp_data['count_vis_resp_fixed'].visual_no_gen))
print('Fixed Visual - Visual No Gen:', exp_fixed_vis_vis_no_gen)

exp_free_vis_vis_no_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_free'].visual, 
                                                   main_exp_data['count_vis_resp_free'].visual_no_gen))
print('Free Visual - Visual No Gen:', exp_free_vis_vis_no_gen)

# Orientation
exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].orientation, 
                                              main_exp_data['count_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)

exp_fixed_free_ori_no_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].orientation_no_gen, 
                                                     main_exp_data['count_vis_resp_free'].orientation_no_gen))
print('Fixed vs Free Orientation No Gen:', exp_fixed_free_ori_no_gen)

exp_fixed_ori_ori_no_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].orientation, 
                                                    main_exp_data['count_vis_resp_fixed'].orientation_no_gen))
print('Fixed Orientation - Orientation No Gen:', exp_fixed_ori_ori_no_gen)

exp_free_ori_ori_no_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_free'].orientation, 
                                                    main_exp_data['count_vis_resp_free'].orientation_no_gen))
print('Free Orientation - Orientation No Gen:', exp_free_ori_ori_no_gen)

# Direction
exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].direction, 
                                              main_exp_data['count_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

exp_fixed_free_dir_no_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].direction_no_gen, 
                                                     main_exp_data['count_vis_resp_free'].direction_no_gen))
print('Fixed vs Free Direction No Gen:', exp_fixed_free_dir_no_gen)

exp_fixed_dir_dir_no_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'].direction, 
                                                    main_exp_data['count_vis_resp_fixed'].direction_no_gen))
print('Fixed Direction - Direction No Gen:', exp_fixed_dir_dir_no_gen)

exp_free_dir_dir_no_gen = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_free'].direction, 
                                                    main_exp_data['count_vis_resp_free'].direction_no_gen))
print('Free Direction - Direction No Gen:', exp_free_dir_dir_no_gen)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_exp_fixed_free_gen_count'] = pd.Series(exp_fixed_free_gen)
    store['kruskal_exp_fixed_free_vis_count'] = pd.Series(exp_fixed_free_vis)
    store['kruskal_exp_fixed_free_vis_no_gen_count'] = pd.Series(exp_fixed_free_vis_no_gen)
    store['kruskal_exp_fixed_vis_vis_no_gen_count'] = pd.Series(exp_fixed_vis_vis_no_gen)
    store['kruskal_exp_free_vis_vis_no_gen_count'] = pd.Series(exp_free_vis_vis_no_gen)
    store['kruskal_exp_fixed_freew_ori_count'] = pd.Series(exp_fixed_free_ori)
    store['kruskal_exp_fixed_free_ori_no_gen_count'] = pd.Series(exp_fixed_free_ori_no_gen)
    store['kruskal_exp_fixed_ori_ori_no_gen_count'] = pd.Series(exp_fixed_ori_ori_no_gen)
    store['kruskal_exp_free_ori_ori_no_gen_count'] = pd.Series(exp_free_ori_ori_no_gen)
    store['kruskal_exp_fixed_free_dir_count'] = pd.Series(exp_fixed_free_dir)
    store['kruskal_exp_fixed_free_dir_no_gen_count'] = pd.Series(exp_fixed_free_dir_no_gen)
    store['kruskal_exp_fixed_dir_dir_no_gen_count'] = pd.Series(exp_fixed_dir_dir_no_gen)
    store['kruskal_exp_free_dir_dir_no_gen_count'] = pd.Series(exp_free_dir_dir_no_gen)

### Wilcoxon Rank Sum

In [ ]:
# Do a Wilcox Rank-Sum test with Bonferroni correction

# General
exp_fixed_free_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].general, 
                                              main_exp_data['count_vis_resp_free'].general))
print('Fixed vs Free General:', exp_fixed_free_gen)

# Visual
exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].visual, 
                                              main_exp_data['count_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

exp_fixed_free_vis_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].visual_no_gen, 
                                                     main_exp_data['count_vis_resp_free'].visual_no_gen))
print('Fixed vs Free Visual No Gen:', exp_fixed_free_vis_no_gen)

exp_fixed_vis_vis_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].visual, 
                                                    main_exp_data['count_vis_resp_fixed'].visual_no_gen))
print('Fixed Visual - Visual No Gen:', exp_fixed_vis_vis_no_gen)

exp_free_vis_vis_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_free'].visual, 
                                                   main_exp_data['count_vis_resp_free'].visual_no_gen))
print('Free Visual - Visual No Gen:', exp_free_vis_vis_no_gen)

# Orientation
exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].orientation, 
                                              main_exp_data['count_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)

exp_fixed_free_ori_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].orientation_no_gen, 
                                                     main_exp_data['count_vis_resp_free'].orientation_no_gen))
print('Fixed vs Free Orientation No Gen:', exp_fixed_free_ori_no_gen)

exp_fixed_ori_ori_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].orientation, 
                                                    main_exp_data['count_vis_resp_fixed'].orientation_no_gen))
print('Fixed Orientation - Orientation No Gen:', exp_fixed_ori_ori_no_gen)

exp_free_ori_ori_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_free'].orientation, 
                                                    main_exp_data['count_vis_resp_free'].orientation_no_gen))
print('Free Orientation - Orientation No Gen:', exp_free_ori_ori_no_gen)

# Direction
exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].direction, 
                                              main_exp_data['count_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

exp_fixed_free_dir_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].direction_no_gen, 
                                                     main_exp_data['count_vis_resp_free'].direction_no_gen))
print('Fixed vs Free Direction No Gen:', exp_fixed_free_dir_no_gen)

exp_fixed_dir_dir_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].direction, 
                                                    main_exp_data['count_vis_resp_fixed'].direction_no_gen))
print('Fixed Direction - Direction No Gen:', exp_fixed_dir_dir_no_gen)

exp_free_dir_dir_no_gen = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_free'].direction, 
                                                    main_exp_data['count_vis_resp_free'].direction_no_gen))
print('Free Direction - Direction No Gen:', exp_free_dir_dir_no_gen)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['wilcoxon_exp_fixed_free_gen_count'] = pd.Series(exp_fixed_free_gen)
    store['wilcoxon_exp_fixed_free_vis_count'] = pd.Series(exp_fixed_free_vis)
    store['wilcoxon_exp_fixed_free_vis_no_gen_count'] = pd.Series(exp_fixed_free_vis_no_gen)
    store['wilcoxon_exp_fixed_vis_vis_no_gen_count'] = pd.Series(exp_fixed_vis_vis_no_gen)
    store['wilcoxon_exp_free_vis_vis_no_gen_count'] = pd.Series(exp_free_vis_vis_no_gen)
    store['wilcoxon_exp_fixed_freew_ori_count'] = pd.Series(exp_fixed_free_ori)
    store['wilcoxon_exp_fixed_free_ori_no_gen_count'] = pd.Series(exp_fixed_free_ori_no_gen)
    store['wilcoxon_exp_fixed_ori_ori_no_gen_count'] = pd.Series(exp_fixed_ori_ori_no_gen)
    store['wilcoxon_exp_free_ori_ori_no_gen_count'] = pd.Series(exp_free_ori_ori_no_gen)
    store['wilcoxon_exp_fixed_free_dir_count'] = pd.Series(exp_fixed_free_dir)
    store['wilcoxon_exp_fixed_free_dir_no_gen_count'] = pd.Series(exp_fixed_free_dir_no_gen)
    store['wilcoxon_exp_fixed_dir_dir_no_gen_count'] = pd.Series(exp_fixed_dir_dir_no_gen)
    store['wilcoxon_exp_free_dir_dir_no_gen_count'] = pd.Series(exp_free_dir_dir_no_gen)

### Mann-Whitney U

In [ ]:
# Do a Mann-Whitney U test with Bonferroni correction

# General
exp_fixed_free_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].general, 
                                              main_exp_data['count_vis_resp_free'].general))
print('Fixed vs Free General:', exp_fixed_free_gen)

# Visual
exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].visual, 
                                              main_exp_data['count_vis_resp_free'].visual))
print('Fixed vs Free Visual:', exp_fixed_free_vis)

exp_fixed_free_vis_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].visual_no_gen, 
                                                     main_exp_data['count_vis_resp_free'].visual_no_gen))
print('Fixed vs Free Visual No Gen:', exp_fixed_free_vis_no_gen)

exp_fixed_vis_vis_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].visual, 
                                                    main_exp_data['count_vis_resp_fixed'].visual_no_gen))
print('Fixed Visual - Visual No Gen:', exp_fixed_vis_vis_no_gen)

exp_free_vis_vis_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_free'].visual, 
                                                   main_exp_data['count_vis_resp_free'].visual_no_gen))
print('Free Visual - Visual No Gen:', exp_free_vis_vis_no_gen)

# Orientation
exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].orientation, 
                                              main_exp_data['count_vis_resp_free'].orientation))
print('Fixed vs Free Orientation:', exp_fixed_free_ori)

exp_fixed_free_ori_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].orientation_no_gen, 
                                                     main_exp_data['count_vis_resp_free'].orientation_no_gen))
print('Fixed vs Free Orientation No Gen:', exp_fixed_free_ori_no_gen)

exp_fixed_ori_ori_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].orientation, 
                                                    main_exp_data['count_vis_resp_fixed'].orientation_no_gen))
print('Fixed Orientation - Orientation No Gen:', exp_fixed_ori_ori_no_gen)

exp_free_ori_ori_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_free'].orientation, 
                                                    main_exp_data['count_vis_resp_free'].orientation_no_gen))
print('Free Orientation - Orientation No Gen:', exp_free_ori_ori_no_gen)

# Direction
exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].direction, 
                                              main_exp_data['count_vis_resp_free'].direction))
print('Fixed vs Free Direction:', exp_fixed_free_dir)

exp_fixed_free_dir_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].direction_no_gen, 
                                                     main_exp_data['count_vis_resp_free'].direction_no_gen))
print('Fixed vs Free Direction No Gen:', exp_fixed_free_dir_no_gen)

exp_fixed_dir_dir_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'].direction, 
                                                    main_exp_data['count_vis_resp_fixed'].direction_no_gen))
print('Fixed Direction - Direction No Gen:', exp_fixed_dir_dir_no_gen)

exp_free_dir_dir_no_gen = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_free'].direction, 
                                                    main_exp_data['count_vis_resp_free'].direction_no_gen))
print('Free Direction - Direction No Gen:', exp_free_dir_dir_no_gen)

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['mannwhitneyu_exp_fixed_free_gen_count'] = pd.Series(exp_fixed_free_gen)
    store['mannwhitneyu_exp_fixed_free_vis_count'] = pd.Series(exp_fixed_free_vis)
    store['mannwhitneyu_exp_fixed_free_vis_no_gen_count'] = pd.Series(exp_fixed_free_vis_no_gen)
    store['mannwhitneyu_exp_fixed_vis_vis_no_gen_count'] = pd.Series(exp_fixed_vis_vis_no_gen)
    store['mannwhitneyu_exp_free_vis_vis_no_gen_count'] = pd.Series(exp_free_vis_vis_no_gen)
    store['mannwhitneyu_exp_fixed_freew_ori_count'] = pd.Series(exp_fixed_free_ori)
    store['mannwhitneyu_exp_fixed_free_ori_no_gen_count'] = pd.Series(exp_fixed_free_ori_no_gen)
    store['mannwhitneyu_exp_fixed_ori_ori_no_gen_count'] = pd.Series(exp_fixed_ori_ori_no_gen)
    store['mannwhitneyu_exp_free_ori_ori_no_gen_count'] = pd.Series(exp_free_ori_ori_no_gen)
    store['mannwhitneyu_exp_fixed_free_dir_count'] = pd.Series(exp_fixed_free_dir)
    store['mannwhitneyu_exp_fixed_free_dir_no_gen_count'] = pd.Series(exp_fixed_free_dir_no_gen)
    store['mannwhitneyu_exp_fixed_dir_dir_no_gen_count'] = pd.Series(exp_fixed_dir_dir_no_gen)
    store['mannwhitneyu_exp_free_dir_dir_no_gen_count'] = pd.Series(exp_free_dir_dir_no_gen)

## Count Vis Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc

# General
count_fixed_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_fixed', 'general',   
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls General 95% CI:', count_fixed_gen[1][1])
print('Fixed vs Controls General Raw:', count_fixed_gen[0])

# Visual
count_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_fixed', 'visual',   
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Visual 95% CI:', count_fixed_vis[1][1])
print('Fixed vs Controls Visual Raw:', count_fixed_vis[0])

count_fixed_vis_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'count_vis_resp_fixed', 'visual_no_gen',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Visual No Gen 95% CI:', count_fixed_vis_no_gen[1][1])
print('Fixed vs Controls Visual No Gen Raw:', count_fixed_vis_no_gen[0])

# Orientation
count_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_fixed', 'orientation',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Orientation 95% CI:', count_fixed_ori[1][1])
print('Fixed vs Controls Orientation Raw:', count_fixed_ori[0])

count_fixed_ori_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'count_vis_resp_fixed', 'orientation_no_gen',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Orientation No Gen 95% CI:', count_fixed_ori_no_gen[1][1])
print('Fixed vs Controls Orientation No Gen Raw:', count_fixed_ori_no_gen[0])


# Direction
count_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_fixed', 'direction',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Direction 95% CI:', count_fixed_dir[1][1])
print('Fixed vs Controls Direction Raw:', count_fixed_dir[0])

count_fixed_dir_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'count_vis_resp_fixed', 'direction_no_gen',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Direction No Gen 95% CI:', count_fixed_dir_no_gen[1][1])
print('Fixed vs Controls Direction No Gen Raw:', count_fixed_dir_no_gen[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_count_fixed_gen'] = pd.DataFrame(list(count_fixed_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_fixed_vis_no_gen'] = pd.DataFrame(list(count_fixed_vis_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_fixed_ori_no_gen'] = pd.DataFrame(list(count_fixed_ori_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_fixed_dir'] = pd.DataFrame(list(count_fixed_dir), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_fixed_dir_no_gen'] = pd.DataFrame(list(count_fixed_dir_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

### Free Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc

# General
count_free_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_free', 'general',   
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls General 95% CI:', count_free_gen[1][1])
print('Free vs Controls General Raw:', count_free_gen[0])

# Visual
count_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_free', 'visual',   
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Visual 95% CI:', count_free_vis[1][1])
print('Free vs Controls Visual Raw:', count_free_vis[0])

count_free_vis_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'count_vis_resp_free', 'visual_no_gen',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Visual No Gen 95% CI:', count_free_vis_no_gen[1][1])
print('Free vs Controls Visual No Gen Raw:', count_free_vis_no_gen[0])

# Orientation
count_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_free', 'orientation',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Orientation 95% CI:', count_free_ori[1][1])
print('Free vs Controls Orientation Raw:', count_free_ori[0])

count_free_ori_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'count_vis_resp_free', 'orientation_no_gen',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Orientation No Gen 95% CI:', count_free_ori_no_gen[1][1])
print('Free vs Controls Orientation No Gen Raw:', count_free_ori_no_gen[0])


# Direction
count_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                 'count_vis_resp_free', 'direction',
                                 kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Direction 95% CI:', count_free_dir[1][1])
print('Free vs Controls Direction Raw:', count_free_dir[0])

count_free_dir_no_gen = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'count_vis_resp_free', 'direction_no_gen',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Direction No Gen 95% CI:', count_free_dir_no_gen[1][1])
print('Free vs Controls Direction No Gen Raw:', count_free_dir_no_gen[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store['kruskal_count_free_gen'] = pd.DataFrame(list(count_free_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_free_vis_no_gen'] = pd.DataFrame(list(count_free_vis_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_free_ori_no_gen'] = pd.DataFrame(list(count_free_ori_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_free_dir'] = pd.DataFrame(list(count_free_dir), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store['kruskal_count_free_dir_no_gen'] = pd.DataFrame(list(count_free_dir_no_gen), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T